To find the depth of flow in a trapezoidal channel using Manning’s equation, you need to use the following steps and formulas:

1. **Manning’s Equation**:

    $ Q = \frac{1}{n} A R^{2/3} S^{1/2} $

    Where:

    - $ Q $ = flow rate (m³/s)
    - $ n $ = Manning’s roughness coefficient
    - $ A $ = cross-sectional area of flow (m²)
    - $ R $ = hydraulic radius (m)
    - $ S $ = slope of the channel (m/m)

2. **Cross-Sectional Area for a Trapezoidal Channel**:

    $A = (b + zy) y$

    Where:

    - $ b $ = bottom width of the channel (m)
    - $ z $ = side slope (horizontal distance per unit vertical distance)
    - $ y $ = depth of flow (m)

3. **Wetted Perimeter for a Trapezoidal Channel**:
   $ P = b + 2y \sqrt{1 + z^2} $

4. **Hydraulic Radius**:
   $ R = \frac{A}{P} $

To find the depth $ y $, follow these steps:

1. Express the cross-sectional area $ A $ and wetted perimeter $ P $ in terms of $ y $:
   $ A = \left(b + zy\right) y $
   $ P = b + 2y \sqrt{1 + z^2} $

2. Substitute these into the hydraulic radius formula:
   $ R = \frac{\left(b + zy\right) y}{b + 2y \sqrt{1 + z^2}} $

3. Substitute $ A $ and $ R $ into Manning’s equation:
   $ Q = \frac{1}{n} \left[\left(b + zy\right) y\right] \left[\frac{\left(b + zy\right) y}{b + 2y \sqrt{1 + z^2}}\right]^{2/3} S^{1/2} $

4. This results in an equation that must be solved iteratively or using numerical methods to find the depth $ y $.

Here procedure for the iterative solution:

1. Assume an initial depth $ y $.
2. Calculate $ A $ using $ A = \left(b + zy\right) y $.
3. Calculate $ P $ using $ P = b + 2y \sqrt{1 + z^2} $.
4. Calculate $ R $ using $ R = \frac{A}{P} $.
5. Substitute $ A $, $ R $, and $ S $ into Manning’s equation to solve for $ Q $.
6. Compare the calculated $ Q $ with the given flow rate.
7. Adjust $ y $ and repeat steps 2-6 until the calculated $ Q $ matches the given $ Q $ within an acceptable tolerance.


In [19]:
import math
import pandas as pd
import argparse

In [20]:
case = {
    1: {'name': "Deep Swale",
        'n': 0.013,
        'b': 0.15,
        'z': 0.225/0.300,
        },
    2: {'name': "Gravel Lane",
        'n': 0.022,
        'b': 0,
        'z': 1/0.035,
        },
}

In [21]:
def manning_trapezoid(Q, n, b, z, S, y_min=0.001, y_max=10, tolerance=1e-6, max_iterations=1000):
    def calculate_flow(y):
        A = (b + z * y) * y
        P = b + 2 * y * math.sqrt(1 + z * z)
        R = A / P
        return (1 / n) * A * (R ** (2/3)) * (S ** 0.5), A

    low, high = y_min, y_max
    for _ in range(max_iterations):
        mid = (low + high) / 2
        Q_mid, A = calculate_flow(mid)

        if abs(Q_mid - Q) < tolerance:
            return mid, A
        elif Q_mid < Q:
            low = mid
        else:
            high = mid
    raise ValueError(
        f"Solution did not converge within {max_iterations} iterations")

In [22]:
def calculate_depth(row):
    if row['Type'] == "Deep Swale":
        params = case[1]
        depth, area = manning_trapezoid(
            row['Peak Discharge'], params['n'], params['b'], params['z'], row['Slope']/100)
    elif row['Type'] == "Gravel Lane":
        params = case[2]
        depth, area = manning_trapezoid(
            row['Peak Discharge'], params['n'], params['b'], params['z'], row['Slope']/100)
    else:
        raise ValueError(f"Unknown type: {row['Type']}")
    return depth, area

In [23]:
def main(input_file):
    data = pd.read_csv(input_file, header=0)

    data[['Flow Depth', 'Area']] = data.apply(
        lambda row: pd.Series(calculate_depth(row)), axis=1)

    data['Flow Velocity'] = data['Peak Discharge'] / data['Area']

    data.drop(['Area'], axis=1, inplace=True)

    new_column_order = ['Location Number', 'Peak Discharge',
                        'Slope', 'Location', 'Type', 'Flow Velocity', 'Flow Depth']

    data = data[new_column_order]

    result_name = input_file.replace('.csv', '_results.csv')

    data.to_csv(result_name, index=False)

    print(f"Results saved to {result_name}")

In [24]:
if __name__ == "__main__":
    # parser = argparse.ArgumentParser(
    #     description='Calculate depth and velocity for channels.')
    # parser.add_argument('input_file', type=str,
    #                     help='Path to the input CSV file')
    # args = parser.parse_args()

    # main(args.input_file)

    main('./data_trapezoid.csv')

Results saved to ./data_trapezoid_results.csv
